In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Bidirectional

from tensorflow.python.keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from numpy.random import seed


SEED = 123  # used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

df = pd.read_csv("reanalysis.csv")

2024-05-15 13:37:48.897564: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 13:37:49.100152: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 13:37:49.685328: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 13:37:51.731769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Convert Categorical column to hot dummy columns
hotencoding1 = pd.get_dummies(df['month'])
hotencoding1 = hotencoding1.add_prefix('month')
hotencoding2 = pd.get_dummies(df['hour'])
hotencoding2 = hotencoding2.add_prefix('hour')

df=df.drop(['month', 'hour'], axis=1)

df=pd.concat([df, hotencoding1, hotencoding2], axis=1)

# Rename response column name for ease of understanding
df=df.rename(columns={'sfcWind':'y'})

#Filter the rows with time less than 2015-01-01
df_train = df[df['time'] < '2013-12-31 23:59:00']
df_test = df[df['time'] >= '2013-12-31 23:59:00']

df_train = df_train.set_index('time')
df_test = df_test.set_index('time')

In [3]:
input_X_train = df_train.loc[:, df_train.columns != 'y'].values  # converts df to numpy array
input_X_test = df_test.loc[:, df_test.columns != 'y'].values  
input_y_train = df_train['y'].values
input_y_test = df_test['y'].values

n_features = input_X_train.shape[1]  # number of features

In [4]:
def temporalize(X, y, lookback):
    '''
    Inputs
    X         A 2D numpy array ordered by time of shape: (n_observations x n_features)
    y         A 1D numpy array with indexes aligned with X, i.e. y[i] should correspond to X[i]. Shape: n_observations.
    lookback  The window size to look back in the past records. Shape: a scalar.

    Output
    output_X  A 3D numpy array of shape: ((n_observations-lookback-1) x lookback x n_features)
    output_y  A 1D array of shape: (n_observations-lookback-1), aligned with X.
    '''
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        output_y.append(y[i + lookback + 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

def flatten(X):
    '''
    Flatten a 3D array.

    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.

    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty(
        (X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1] - 1), :]
    return flattened_X

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize

    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])

    return X

In [5]:
lookback = 5
X_train, y_train = temporalize(X=input_X_train, 
                      y=input_y_train, 
                      lookback=lookback)
X_test, y_test = temporalize(X=input_X_test, 
                      y=input_y_test, 
                      lookback=lookback)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=DATA_SPLIT_PCT,
    random_state=SEED)

TIMESTEPS = X_train.shape[1]  # equal to the lookback
N_FEATURES = X_train.shape[2]  # the number of features

In [6]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(X_train))
X_train_scaled = scale(X_train, scaler)
X_valid_scaled = scale(X_valid, scaler)
X_test_scaled =  scale(X_test, scaler)

# Modelo

In [7]:
model = Sequential()
model.add(Input(shape=(TIMESTEPS, N_FEATURES), 
                name='input'))
model.add(
    LSTM(units=16, 
         activation='tanh',
         recurrent_activation='sigmoid',
         return_sequences=True, 
         name='lstm_layer_1'))
model.add(
    LSTM(units=8, 
         activation='tanh', 
         recurrent_activation='sigmoid',
         return_sequences=False, 
         name='lstm_layer_2'))
model.add(Dense(units=1, 
                activation='linear', 
                name='output'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_layer_1 (LSTM)             │ (None, 5, 16)          │         3,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer_2 (LSTM)             │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,777 (18.66 KB)

 Trainable params: 4,777 (18.66 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])

In [9]:
history = model.fit(x= np.asarray(X_train_scaled).astype('float32'),
                    y=y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                     y_valid),
                    verbose=1).history

Epoch 1/100


2024-05-15 13:38:20.390650: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 88279200 exceeds 10% of free system memory.


In [ ]:
p = model.predict(np.asarray(X_test_scaled).astype('float32'))
res = pd.DataFrame(p)
res.to_csv("lstm_output.csv", index=False)